# 🦴 Vertebrae Segmentation Pipeline - Kaggle Training

This notebook trains the complete 3-stage pipeline using **CLI commands**.

## Steps:
1. Clone repository from GitHub
2. Install dependencies
3. Run training commands for each stage
4. Export trained models

In [ ]:
# Install required packages
!pip install SimpleITK nibabel tensorboard scipy pandas scikit-learn tqdm --quiet

import sys
import os
import torch

print(f"Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# ============================================================
# CONFIGURE YOUR GITHUB REPOSITORY
# ============================================================
GITHUB_REPO = "https://github.com/YOUR_USERNAME/YOUR_REPO.git"  # <-- CHANGE THIS

REPO_NAME = GITHUB_REPO.split('/')[-1].replace('.git', '')

%cd /kaggle/working

# Clone if not exists
if not os.path.exists(REPO_NAME):
    !git clone {GITHUB_REPO}
    print(f"✅ Cloned {REPO_NAME}")
else:
    print(f"✅ Repository exists, pulling latest...")
    %cd {REPO_NAME}
    !git pull
    %cd ..

# Add to Python path
PIPELINE_DIR = f"/kaggle/working/{REPO_NAME}"
sys.path.insert(0, PIPELINE_DIR)

# Add to sys.path for imports
sys.path.insert(0, PIPELINE_DIR)

!ls {REPO_NAME}/
# Verify

## ⚠️ Important: Commit Changes to GitHub

Before running on Kaggle, make sure you've committed the updated `train.py` and `infer.py` files to GitHub.

These files have been updated to use **absolute imports** instead of relative imports, which is required for the code to run properly.

```bash
git add train.py infer.py
git commit -m "Fix imports for direct script execution"
git push origin main
```

In [ ]:
# ============================================================
# CONFIGURE PATHS
# ============================================================
DATASET_CSV = "/kaggle/input/verse19/dataset.csv"  # <-- CHANGE THIS
OUTPUT_DIR = "/kaggle/working/outputs"
FOLD = 0
NUM_FOLDS = 5
DEVICE = "cuda"

# Training epochs
EPOCHS_STAGE1 = 100
EPOCHS_STAGE2 = 150
EPOCHS_STAGE3 = 100
BATCH_SIZE = 2
LR = 0.0001

# ============================================================
# QUICK TEST MODE
# ============================================================
QUICK_TEST = True  # <-- Set to False for full training

if QUICK_TEST:
    EPOCHS_STAGE1 = 3
    EPOCHS_STAGE2 = 3
    EPOCHS_STAGE3 = 3
    print("⚠️ QUICK TEST MODE: 3 epochs per stage")

os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Dataset: {DATASET_CSV}")
print(f"Output: {OUTPUT_DIR}")

---
# 🚀 Stage 1: Spine Localization

In [ ]:
%cd {PIPELINE_DIR}

print("="*70)
print("STAGE 1: SPINE LOCALIZATION")
print("="*70)

# Import and run training
sys.path.insert(0, PIPELINE_DIR)
import train

# Set command-line arguments
sys.argv = [
    'train.py',
    '--stage', 'spine',
    '--csv', DATASET_CSV,
    '--output', OUTPUT_DIR,
    '--fold', str(FOLD),
    '--num_folds', str(NUM_FOLDS),
    '--epochs', str(EPOCHS_STAGE1),
    '--batch_size', str(BATCH_SIZE),
    '--lr', str(LR),
    '--device', DEVICE
]

# Run training
train.main()

print("\n✅ Stage 1 Complete!")

---
# 🚀 Stage 2: Vertebrae Localization

In [ ]:
print("="*70)
print("STAGE 2: VERTEBRAE LOCALIZATION")
print("="*70)

# Set command-line arguments
sys.argv = [
    'train.py',
    '--stage', 'vertebrae',
    '--csv', DATASET_CSV,
    '--output', OUTPUT_DIR,
    '--fold', str(FOLD),
    '--num_folds', str(NUM_FOLDS),
    '--epochs', str(EPOCHS_STAGE2),
    '--batch_size', str(BATCH_SIZE),
    '--lr', str(LR),
    '--device', DEVICE
]

# Run training
train.main()

print("\n✅ Stage 2 Complete!")

---
# 🚀 Stage 3: Vertebrae Segmentation

In [ ]:
print("="*70)
print("STAGE 3: VERTEBRAE SEGMENTATION")
print("="*70)

# Set command-line arguments
sys.argv = [
    'train.py',
    '--stage', 'segmentation',
    '--csv', DATASET_CSV,
    '--output', OUTPUT_DIR,
    '--fold', str(FOLD),
    '--num_folds', str(NUM_FOLDS),
    '--epochs', str(EPOCHS_STAGE3),
    '--batch_size', str(BATCH_SIZE),
    '--lr', str(LR),
    '--device', DEVICE
]

# Run training
train.main()

print("\n✅ Stage 3 Complete!")

---
# 🧪 Test Inference

In [ ]:
import pandas as pd
df = pd.read_csv(DATASET_CSV)
TEST_IMAGE = df.iloc[0]['image_path']

print(f"Testing inference on: {TEST_IMAGE}")

# Import and run inference
import infer

# Set command-line arguments
sys.argv = [
    'infer.py',
    '--image', TEST_IMAGE,
    '--model_dir', OUTPUT_DIR,
    '--output', f'{OUTPUT_DIR}/inference_test',
    '--device', DEVICE,
    '--save_intermediates'
]

# Run inference
infer.main()

print("\n✅ Inference Complete!")

In [ ]:
import shutil

zip_path = f"/kaggle/working/trained_models_fold{FOLD}"
shutil.make_archive(zip_path, 'zip', OUTPUT_DIR)

print(f"✅ Created: {zip_path}.zip")
print(f"📥 Download from Output tab!")